In [1]:
#!pip install pysam
import pysam

In [19]:
samfile = pysam.AlignmentFile("merged-tumor.bam")
first_read = next(samfile)
print(first_read)

C0HVYACXX120402:7:1207:5722:57044	1187	#20	9483249	27	76M	#20	9483382	209	TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG	array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34])	[('XA', 'GL000217.1,-110754,76M,1;'), ('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('MD', '76'), ('RG', '1'), ('BI', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NM', 0), ('MQ', 27), ('AS', 76), ('XS', 71)]


In [16]:
print('First read name: ', first_read.query_name)
print('Flag: ', first_read.flag)
print('Reference(contig) id:', first_read.reference_id)
print('POS (leftmost coordinate, 0-based): ', first_read.reference_start)
print('Mapping quality: ', first_read.mapping_quality)
print('CIGAR string: ', first_read.cigarstring)
print('RNEXT (reference name of the next/mate read): ', first_read.next_reference_id)
print('PNEXT (position of the mate/next read): ', first_read.next_reference_start)
print('Sequence: ', first_read.query_sequence)
print('Quality seq: ', first_read.qual)

First read name:  C0HVYACXX120402:7:1207:5722:57044
Flag:  1187
Reference(contig) id: 20
POS (leftmost coordinate, 0-based):  9483248
Mapping quality:  27
CIGAR string:  76M
RNEXT (reference name of the next/mate read):  20
PNEXT (position of the mate/next read):  9483381
Sequence:  TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG
Quality seq:  ==<>@?@@>@D?>@C?>8JA5?>CC>?@???BB;H-:4A?D=DB8BDE?GBJCD@B8??E<A>CDJB@B>AA@@@C


First read: flag = 1187 = 10010100011b

1-This is a paired read.

1-This read is part of a pair that aligned properly. Proper alignment indicates both reads in a pair are oriented towards one another (one forward, one reverse), are both on the same contig, and are within the expected distance from one another.

0-This read was aligned

0-This read is part of a pair and its mate is aligned

0-This read is not aligned in the reverse direction

1-This read is part of a pair and its mate aligned in the reverse direction

0-This read is not the first in the pair (read1)

1-This read is the second in pair (read2)

0-The given alignment is a not secondary alignment; The read didn't have multiple potential alignments

0-Read didn't fail quality check

1-Read is a duplicate (PCR or optical duplicate)

In [20]:
for i in range(20):
    read = next(samfile.fetch(until_eof = True))
    print(read.flag)

163
99
99
99
163
99
1123
99
1123
99
163
1187
163
147
163
163
163
163
163
99


163 = 10100011b
Same as the first read, only this is not a duplicate

99 = 1100011b
This is a paired read, that is also a part of pair that aligned properly.
Its mate aligned in the reverse direction.
This is a first mate (read1).


1123 = 10001100011b
This is a paired read, that is also a part of pair that aligned properly.
Its mate aligned in the reverse direction.
This is a first mate (read1).
This is a duplicate read.


147 = 10010011b
This is a paired read, that is also a part of pair that aligned properly.
Read is aligned in reverse direction.
Read is a second in pair (read2).

In [25]:
samfile.close()

In [34]:
unmapped_cnt = 0
zero_mapq_cnt = 0
mapq_sum = 0
nonzero_mapq_sum = 0
cnt = 0

for read in pysam.AlignmentFile("merged-tumor.bam"):
    cnt += 1
    if read.mapping_quality != 255:
        mapq_sum += read.mapping_quality
        
    if read.is_unmapped:
        unmapped_cnt += 1
    if read.mapping_quality == 0:
        zero_mapq_cnt += 1
    else:
        if read.mapping_quality != 255:
            nonzero_mapq_sum += read.mapping_quality
    
print('Number of unmapped reads: ', unmapped_cnt)
print('Number of reads with mapping quality 0: ', zero_mapq_cnt)
print('Total number of reads: ', cnt)
print('Average mapping quality for all reads: ', float(mapq_sum) / cnt)
print('Average mapping quality for nonzero mapping quality reads: ', float (nonzero_mapq_sum) / (cnt - zero_mapq_cnt))


Number of unmapped reads:  17765
Number of reads with mapping quality 0:  126628
Total number of reads:  2921629
Average mapping quality for all reads:  55.91379158681681
Average mapping quality for nonzero mapping quality reads:  58.446975510921106


In [36]:

# This could be done by making a list of AlignedSegments with almost three million elements, and using Pythons
# built-in functions, but this would mean to have whole list loaded which could use a lot of memory
unmapped_cnt = 0
zero_mapq_cnt = 0
mapq_sum = 0
nonzero_mapq_sum = 0
cnt = 0

l = []
for read in pysam.AlignmentFile("merged-tumor.bam"):
    l.append(read)

cnt = len(l)
unmapped_cnt = sum(map(lambda x : x.is_unmapped, l))
zero_mapq_cnt = sum(map(lambda x : x.mapping_quality == 0, l))
mapq_sum = sum(r.mapping_quality for r in l)
nonzero_mapq_sum = sum(r.mapping_quality for r in l if r.mapping_quality > 0)

print('Number of unmapped reads: ', unmapped_cnt)
print('Number of reads with mapping quality 0: ', zero_mapq_cnt)
print('Total number of reads: ', cnt)
print('Average mapping quality for all reads: ', float(mapq_sum) / cnt)
print('Average mapping quality for nonzero mapping quality reads: ', float (nonzero_mapq_sum) / (cnt - zero_mapq_cnt))


Number of unmapped reads:  17765
Number of reads with mapping quality 0:  126628
Total number of reads:  2921629
Average mapping quality for all reads:  55.91379158681681
Average mapping quality for nonzero mapping quality reads:  58.446975510921106
